In [0]:
%fs
ls FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/Streaming_History_Audio_2019_2024.json,Streaming_History_Audio_2019_2024.json,10346066,1714233022000


In [0]:
dbutils.fs.ls('dbfs:/FileStore/tables')

[FileInfo(path='dbfs:/FileStore/tables/Streaming_History_Audio_2019_2024.json', name='Streaming_History_Audio_2019_2024.json', size=10346066, modificationTime=1714233022000)]

In [0]:
df = spark.read.option('multiline', 'true').json('dbfs:/FileStore/tables/Streaming_History_Audio_2019_2024.json')

In [0]:
df.printSchema()

root
 |-- conn_country: string (nullable = true)
 |-- episode_name: string (nullable = true)
 |-- episode_show_name: string (nullable = true)
 |-- incognito_mode: boolean (nullable = true)
 |-- ip_addr_decrypted: string (nullable = true)
 |-- master_metadata_album_album_name: string (nullable = true)
 |-- master_metadata_album_artist_name: string (nullable = true)
 |-- master_metadata_track_name: string (nullable = true)
 |-- ms_played: long (nullable = true)
 |-- offline: boolean (nullable = true)
 |-- offline_timestamp: long (nullable = true)
 |-- platform: string (nullable = true)
 |-- reason_end: string (nullable = true)
 |-- reason_start: string (nullable = true)
 |-- shuffle: boolean (nullable = true)
 |-- skipped: boolean (nullable = true)
 |-- spotify_episode_uri: string (nullable = true)
 |-- spotify_track_uri: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- user_agent_decrypted: string (nullable = true)
 |-- username: string (nullable = true)



In [0]:
# No. of Columns in df dataset
len(df.columns)

21

In [0]:
# Total no of records in dataset.
df.count()

13569

In [0]:
df.show(5)

+------------+------------+-----------------+--------------+-----------------+--------------------------------+---------------------------------+--------------------------+---------+-------+-----------------+--------------------+----------+------------+-------+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|conn_country|episode_name|episode_show_name|incognito_mode|ip_addr_decrypted|master_metadata_album_album_name|master_metadata_album_artist_name|master_metadata_track_name|ms_played|offline|offline_timestamp|            platform|reason_end|reason_start|shuffle|skipped|spotify_episode_uri|   spotify_track_uri|                  ts|user_agent_decrypted|            username|
+------------+------------+-----------------+--------------+-----------------+--------------------------------+---------------------------------+--------------------------+---------+-------+-----------------+--------------------+----------+------------

In [0]:
df = df.withColumnRenamed('master_metadata_album_album_name' ,'album_name')\
    .withColumnRenamed('master_metadata_album_artist_name', 'artist_name')\
        .withColumnRenamed('master_metadata_track_name', 'track_name')
df.show(10)

+------------+------------+-----------------+--------------+-----------------+--------------------+------------------+--------------------+---------+-------+-----------------+--------------------+----------+------------+-------+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|conn_country|episode_name|episode_show_name|incognito_mode|ip_addr_decrypted|          album_name|       artist_name|          track_name|ms_played|offline|offline_timestamp|            platform|reason_end|reason_start|shuffle|skipped|spotify_episode_uri|   spotify_track_uri|                  ts|user_agent_decrypted|            username|
+------------+------------+-----------------+--------------+-----------------+--------------------+------------------+--------------------+---------+-------+-----------------+--------------------+----------+------------+-------+-------+-------------------+--------------------+--------------------+--------------------

In [0]:
df.select('platform').distinct().show()

+--------------------+
|            platform|
+--------------------+
|             android|
|Android-tablet OS...|
|web_player window...|
|Android-tablet OS...|
|web_player window...|
|Android OS 10 API...|
|web_player window...|
|web_player window...|
|Android OS 11 API...|
|Windows 10 (10.0....|
|web_player window...|
|Android OS 12 API...|
|web_player window...|
|             windows|
+--------------------+



In [0]:
from pyspark.sql.functions import lower, when
df = df.withColumn('platform',
                   when(lower('platform').contains('windows'), 'Windows')\
                       .when(lower('platform').contains('android'), 'Android')\
                           .otherwise(None)
                           )
df.show(10)

+------------+------------+-----------------+--------------+-----------------+--------------------+------------------+--------------------+---------+-------+-----------------+--------+----------+------------+-------+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|conn_country|episode_name|episode_show_name|incognito_mode|ip_addr_decrypted|          album_name|       artist_name|          track_name|ms_played|offline|offline_timestamp|platform|reason_end|reason_start|shuffle|skipped|spotify_episode_uri|   spotify_track_uri|                  ts|user_agent_decrypted|            username|
+------------+------------+-----------------+--------------+-----------------+--------------------+------------------+--------------------+---------+-------+-----------------+--------+----------+------------+-------+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|          IN

In [0]:
# Creating a new dataset by considering only required features.
sp_df = df.select(['album_name', 'artist_name', 'track_name', 'ms_played', 'platform', 'reason_end', 'reason_start', 'shuffle', 'skipped', 'ts'])

In [0]:
len(sp_df.columns)

10

In [0]:
# No. of records in dataset
sp_df.count()

13569

In [0]:
sp_df.show(10)

+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+
|          album_name|       artist_name|          track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped|                  ts|
+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+
|So Far Away (feat...|     Martin Garrix|So Far Away (feat...|    31118| Android|   endplay|    clickrow|  false|   NULL|2019-05-01T03:45:22Z|
|           Boomerang|            Brooks|           Boomerang|        0| Android|   endplay|    clickrow|  false|   NULL|2019-05-01T03:45:22Z|
|           Boomerang|            Brooks|           Boomerang|    24928| Android|   backbtn|    clickrow|  false|   NULL|2019-05-01T03:45:48Z|
|               Seven|     Martin Garrix|Sun Is Never Goin...|     1960| Android|   endplay|     backbtn|  false|   NULL|2019-05-01T03:46:03Z|

In [0]:
#Displaying the count of NULL values in each Column.
from pyspark.sql.functions import col, when, count, coalesce
df_null_count = sp_df.select([count(when(col(c).isNull(),c)).alias(c) for c in sp_df.columns])
df_null_count.show(10)
# A Dictonary with columns and count of null values.
df_null_count_dict = {c : sp_df.filter(col(c).isNull()).count() for c in sp_df.columns}
df_null_count_dict

+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+
|album_name|artist_name|track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped| ts|
+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+
|        12|         12|        12|        0|       0|         0|           0|      0|   1073|  0|
+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+



{'album_name': 12,
 'artist_name': 12,
 'track_name': 12,
 'ms_played': 0,
 'platform': 0,
 'reason_end': 0,
 'reason_start': 0,
 'shuffle': 0,
 'skipped': 1073,
 'ts': 0}

In [0]:
sp_df = sp_df.dropna(subset=['album_name', 'artist_name', 'track_name'])

In [0]:
# Replacing the NULL values to False
#Method 1 - Using WHEN function.
sp_df = sp_df.withColumn('skipped', when(col('skipped').isNull(), False).otherwise(col('skipped')))

#Method 2 - Using COALESCE function.
# sp_df = sp_df.withColumn('skipped', coalesce(col('skipped'), lit(False)))

In [0]:
from pyspark.sql.functions import col, when, count, coalesce
df_null_count = sp_df.select([count(when(col(c).isNull(),c)).alias(c) for c in sp_df.columns])
df_null_count.show(10)

+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+
|album_name|artist_name|track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped| ts|
+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+
|         0|          0|         0|        0|       0|         0|           0|      0|      0|  0|
+----------+-----------+----------+---------+--------+----------+------------+-------+-------+---+



In [0]:
sp_df = sp_df.withColumn('date', col('ts').cast('date'))
sp_df.show(10)

+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+
|          album_name|       artist_name|          track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped|                  ts|      date|
+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+
|So Far Away (feat...|     Martin Garrix|So Far Away (feat...|    31118| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|
|           Boomerang|            Brooks|           Boomerang|        0| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|
|           Boomerang|            Brooks|           Boomerang|    24928| Android|   backbtn|    clickrow|  false|  false|2019-05-01T03:45:48Z|2019-05-01|
|               Seven|     Martin Garrix|Sun Is Never Goin...|     1960| And

In [0]:
from pyspark.sql.functions import date_format
sp_df = sp_df.withColumn('year', date_format('date', 'y'))\
        .withColumn('month', date_format('date', 'MMMM'))\
            .withColumn('day', date_format('date', 'EEEE'))
sp_df.show(10)

+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+----+-----+---------+
|          album_name|       artist_name|          track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped|                  ts|      date|year|month|      day|
+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+----+-----+---------+
|So Far Away (feat...|     Martin Garrix|So Far Away (feat...|    31118| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|2019|  May|Wednesday|
|           Boomerang|            Brooks|           Boomerang|        0| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|2019|  May|Wednesday|
|           Boomerang|            Brooks|           Boomerang|    24928| Android|   backbtn|    clickrow|  false|  false|2019

In [0]:
from pyspark.sql.types import IntegerType
sp_df = sp_df.withColumn('month_num', date_format(col('ts'), 'M'))
sp_df = sp_df.withColumn('day_num',
                 when(col('day')=='Sunday', 1)\
                     .when(col('day')=='Monday', 2)\
                         .when(col('day')=='Tuesday', 3)\
                             .when(col('day')=='Wednesday', 4)\
                                 .when(col('day')=='Thursday', 5)\
                                     .when(col('day')=='Friday', 6)\
                                         .when(col('day')=='Saturday', 7)\
                                             .otherwise(None))
sp_df = sp_df.withColumn('hour', date_format(col('ts'), 'H'))
sp_df = sp_df.withColumn('year', col('year').cast(IntegerType()))\
    .withColumn('month_num', col('month_num').cast(IntegerType()))\
        .withColumn('hour', col('hour').cast(IntegerType()))
sp_df.show(10)

+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+----+-----+---------+---------+-------+----+
|          album_name|       artist_name|          track_name|ms_played|platform|reason_end|reason_start|shuffle|skipped|                  ts|      date|year|month|      day|month_num|day_num|hour|
+--------------------+------------------+--------------------+---------+--------+----------+------------+-------+-------+--------------------+----------+----+-----+---------+---------+-------+----+
|So Far Away (feat...|     Martin Garrix|So Far Away (feat...|    31118| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|2019|  May|Wednesday|        5|      4|   3|
|           Boomerang|            Brooks|           Boomerang|        0| Android|   endplay|    clickrow|  false|  false|2019-05-01T03:45:22Z|2019-05-01|2019|  May|Wednesday|        5|      4|   3|
|         

In [0]:
sp_df.printSchema()

root
 |-- album_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- ms_played: long (nullable = true)
 |-- platform: string (nullable = true)
 |-- reason_end: string (nullable = true)
 |-- reason_start: string (nullable = true)
 |-- shuffle: boolean (nullable = true)
 |-- skipped: boolean (nullable = true)
 |-- ts: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month_num: integer (nullable = true)
 |-- day_num: integer (nullable = true)
 |-- hour: integer (nullable = true)



In [0]:
sp_df.groupBy('reason_end').count().orderBy('count', ascending = False).show()
sp_df.groupBy('shuffle').count().orderBy('count', ascending = False).show()

+--------------------+-----+
|          reason_end|count|
+--------------------+-----+
|              fwdbtn| 4593|
|             endplay| 4420|
|           trackdone| 3429|
|              logout|  510|
|unexpected-exit-w...|  379|
|             backbtn|  162|
|     unexpected-exit|   45|
|              remote|   16|
|             unknown|    2|
|          trackerror|    1|
+--------------------+-----+

+-------+-----+
|shuffle|count|
+-------+-----+
|  false| 7842|
|   true| 5715|
+-------+-----+



In [0]:
sp_df.write.saveAsTable('spotify', format='delta', mode='overwrite')